In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import copy, torch, os
import pandas as pd
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
data_df = pd.concat([train_df, test_df], ignore_index=True, sort=False)
data_df = data_df.drop(columns=['Descript','Resolution','DayOfWeek','Address'])
print(data_df.columns.values)

['Dates' 'Category' 'PdDistrict' 'X' 'Y' 'Id']


# Data Cleaning

In [2]:
print ("{:<15} {:<15} {:<15}".format('Coloums','Has null','Dtype'))
for i in data_df.columns:
    print ("{:<15} {:<15}".format(i,data_df[i].isnull().sum()),data_df[i].dtype)

Coloums         Has null        Dtype          
Dates           0               object
Category        884262          object
PdDistrict      0               object
X               0               float64
Y               0               float64
Id              878049          float64


The data has mixed value in dates to adresss

1. Create dates feature to year, month, hour, day in the week

In [3]:
data_df['Dates'] = pd.to_datetime(data_df['Dates'])
data_df['Year'] = data_df['Dates'].dt.year
data_df['Month'] = data_df['Dates'].dt.month
data_df['Hour'] = data_df['Dates'].dt.hour
data_df['DayOfWeek'] = data_df['Dates'].dt.dayofweek
data_df['Day'] = data_df['Dates'].dt.day
data_df = data_df.drop(columns='Dates')
from sklearn import preprocessing
label_list = ['Category','PdDistrict']
le = []
for list,i in enumerate(label_list):
    le.append(preprocessing.LabelEncoder())
    le[list].fit(data_df[i])
    data_df[i] = le[list].transform(data_df[i])
temp = [print(index,val) for index, val in enumerate(le[0].classes_)]

0 ARSON
1 ASSAULT
2 BAD CHECKS
3 BRIBERY
4 BURGLARY
5 DISORDERLY CONDUCT
6 DRIVING UNDER THE INFLUENCE
7 DRUG/NARCOTIC
8 DRUNKENNESS
9 EMBEZZLEMENT
10 EXTORTION
11 FAMILY OFFENSES
12 FORGERY/COUNTERFEITING
13 FRAUD
14 GAMBLING
15 KIDNAPPING
16 LARCENY/THEFT
17 LIQUOR LAWS
18 LOITERING
19 MISSING PERSON
20 NON-CRIMINAL
21 OTHER OFFENSES
22 PORNOGRAPHY/OBSCENE MAT
23 PROSTITUTION
24 RECOVERED VEHICLE
25 ROBBERY
26 RUNAWAY
27 SECONDARY CODES
28 SEX OFFENSES FORCIBLE
29 SEX OFFENSES NON FORCIBLE
30 STOLEN PROPERTY
31 SUICIDE
32 SUSPICIOUS OCC
33 TREA
34 TRESPASS
35 VANDALISM
36 VEHICLE THEFT
37 WARRANTS
38 WEAPON LAWS
39 nan


In [4]:
data_df['Category'] = np.where(data_df['Category']==39,np.nan,data_df['Category'])

In [5]:
(data_df[data_df['Category']>=0]).drop(columns=['Id','Category'])

,PdDistrict,X,Y,Year,Month,Hour,DayOfWeek,Day
0,4,-122.425892,37.774599,2015,5,23,2,13
1,4,-122.425892,37.774599,2015,5,23,2,13
2,4,-122.424363,37.800414,2015,5,23,2,13
3,4,-122.426995,37.800873,2015,5,23,2,13
4,5,-122.438738,37.771541,2015,5,23,2,13
...,...,...,...,...,...,...,...,...
878044,8,-122.459033,37.714056,2003,1,0,0,6
878045,2,-122.447364,37.731948,2003,1,0,0,6
878046,7,-122.403390,37.780266,2003,1,0,0,6
878047,7,-122.390531,37.780607,2003,1,0,0,6


In [6]:
train_data = (data_df[data_df['Category']>=0]).drop(columns=['Id'])
X_train = train_data.drop(columns=['Category'])
y_train = train_data['Category']

y_test = (data_df[data_df['Category'].isnull()]).drop(columns=['Id','Category'])

In [7]:
from sklearn.model_selection import train_test_split
X_train0, X_train1, y_train0, y_train1 = train_test_split(X_train, y_train, 
                                                test_size=0.2, random_state=0)

In [13]:
from sklearn.tree import DecisionTreeClassifier
DecisionTreeClassifierModel = DecisionTreeClassifier()
DecisionTreeClassifierModel.fit(X_train0, y_train0)

DecisionTreeClassifier()

In [9]:
from sklearn.naive_bayes import GaussianNB
GaussianNBModel = GaussianNB()
GaussianNBModel.fit(X_train0, y_train0)


0.07401923868122356

In [12]:
GaussianNBModel.predict(X_train1).max()

36.0